In [1]:
import pandas as pd

# === Configura esto ===
RUTA_CSV = "phishing_data_es.csv"     # <- tu archivo
SEP = None                  # None = que pandas lo infiera (o pon ";" / "," ...)
ENCODING = None             # Ej: "utf-8" o "latin-1"
DTYPE_AS_STR = True         # Forzar a str para evitar NaN/objetos raros

read_kwargs = dict(sep=SEP, encoding=ENCODING, engine="python")
if DTYPE_AS_STR:
    read_kwargs["dtype"] = str

df = pd.read_csv(RUTA_CSV, **read_kwargs)
df.head()


,subject,body,label,source_file
0,ASISTENCIA A LAS EMPRESAS URGENTES Y ASOCIACIÓN,DE:MR. James NGOLA. TEL. CONFIDENCIAL: 233-27-...,1,Nigerian_Fraud_es.csv
1,ASISTENCIA URGENTE / RELACIÓN (P),"Querido amigo, soy el Sr. Ben Suleman un Offic...",1,Nigerian_Fraud_es.csv
2,Buenos días a ti.,DE SU MAJESTAD ROYAL (HRM) RÚBRICA CROWN DEL R...,1,Nigerian_Fraud_es.csv
3,Buenos días a ti.,DE SU MAJESTAD ROYAL (HRM) RÚBRICA CROWN DEL R...,1,Nigerian_Fraud_es.csv
4,Necesito tu ayuda.,"Estimado señor, Es con un corazón lleno de esp...",1,Nigerian_Fraud_es.csv


In [2]:
import re

# Parámetros del detector
THRESHOLD = 10                 # Longitud mínima de la racha
CHARS = "*=+-_'.¡! "               # Conjunto de caracteres a considerar (incluye espacio)

def build_pattern(chars: str, threshold: int) -> re.Pattern:
    charclass = ''.join(re.escape(c) for c in chars)
    return re.compile(fr'[{charclass}]{{{threshold},}}')

def find_garbage_rows(df: pd.DataFrame, pattern: re.Pattern, cols=None) -> pd.Series:
    """
    Devuelve una Serie booleana (True = fila con 'basura').

    cols:
      - None -> usa todas las columnas (convierte a str y une por espacio)
      - lista de columnas -> solo revisa esas columnas
    """
    if cols is not None:
        row_text = df[cols].astype(str).fillna('').agg(' '.join, axis=1)
    else:
        row_text = df.astype(str).fillna('').agg(' '.join, axis=1)
    return row_text.str.contains(pattern)

pattern = build_pattern(CHARS, THRESHOLD)


In [3]:
mask = find_garbage_rows(df, pattern, cols=["body"])

total = len(df)
malos = int(mask.sum())
pct = (malos / total * 100) if total else 0.0

print(f"Total de filas: {total}")
print(f"Filas con basura (≥ {THRESHOLD} de solo '{CHARS}'): {malos} ({pct:.2f}%)")

# Mostrar algunos ejemplos detectados
ejemplos = df[mask].head(10)
ejemplos


Total de filas: 75112
Filas con basura (≥ 10 de solo '*=+-_'.¡! '): 17799 (23.70%)


,subject,body,label,source_file
19,SE BUSCA PARA FRAUDE: Fawwaz Ulaby,$250.000 PARA INFORMACIÓN: LLEGANDO AL INDICAC...,1,Nigerian_Fraud_es.csv
49,NaN,"Estimado señor, FIRSt Debo solicitar su confia...",1,Nigerian_Fraud_es.csv
59,URGENTE Y CONFIDENCIAL,DESDE:Dopp Bob. ''''''''''''''''''''''''''''''...,1,Nigerian_Fraud_es.csv
75,Asistencia urgente.,"Estimado señor, tengo una propuesta para usted...",1,Nigerian_Fraud_es.csv
79,PRÓXIMO DE LA KIN,"ATTN:PRESIDENTE Soy BARRISTER JOSEPH BENSON, e...",1,Nigerian_Fraud_es.csv
81,urgente,PROPUESTA DE EMPRESAS CONFIDENTALES >DEPENDIEN...,1,Nigerian_Fraud_es.csv
96,"Por favor, venga y conozca a su Dios y también...","Por favor, venid y ayudad Antes de leer mi men...",1,Nigerian_Fraud_es.csv
99,PROPUESTA DE NEGOCIACIÓN.,BARR. FEMI WILLIAMS TEL: 234-803-320-5430 FAX:...,1,Nigerian_Fraud_es.csv
124,Asistencia empresarial urgente.,No 13 VS2 Cethswayo Estate Generation-South Af...,1,Nigerian_Fraud_es.csv
132,URGENTE Y CONFIDENCIAL,No.13 VS2 Cethswayo Estate Generation-South Af...,1,Nigerian_Fraud_es.csv


In [4]:
GUARDAR_A = "filas_basura.csv"   # o None para no guardar

if GUARDAR_A:
    df[mask].to_csv(GUARDAR_A, index=False)
    print(f"Guardado: {GUARDAR_A}")


Guardado: filas_basura.csv


In [5]:
# 5) Limpiar y guardar (eliminando las rachas detectadas)

import re
import pandas as pd

# Usa los mismos THRESHOLD y CHARS definidos antes
# THRESHOLD = 10
# CHARS = "*=+-_ "   # incluye espacio; quítalo si no quieres borrar espacios

# === Parámetros de limpieza ===
COLS_A_LIMPIAR = ["body"]        # None = todas las columnas; o lista, ej.: ["descripcion"]
REEMPLAZO = " "               # "" para eliminar, " " para reemplazar por un espacio
NORMALIZAR_ESPACIOS = True   # Colapsar espacios múltiples y hacer strip
GUARDAR_LIMPIO_A = "datos_limpios.csv"

# Compilar patrón (≥ THRESHOLD consecutivos solo de CHARS)
charclass = ''.join(re.escape(c) for c in CHARS)
garbage_re = re.compile(fr'[{charclass}]{{{THRESHOLD},}}')

def limpiar_dataframe(df: pd.DataFrame,
                      pattern: re.Pattern,
                      cols=None,
                      reemplazo: str = "",
                      normalizar_espacios: bool = True) -> pd.DataFrame:
    df2 = df.copy()
    target_cols = (df2.columns if cols is None else cols)

    # Si tu df fue leído como strings, esto es directo; si no, se convierte a str por columna
    for col in target_cols:
        df2[col] = df2[col].astype(str).str.replace(pattern, reemplazo, regex=True)
        if normalizar_espacios:
            # Colapsa espacios en blanco repetidos y hace strip
            df2[col] = df2[col].str.replace(r"\s{2,}", " ", regex=True).str.strip()
    return df2

# Limpiar
df_limpio = limpiar_dataframe(df, garbage_re,
                              cols=COLS_A_LIMPIAR,
                              reemplazo=REEMPLAZO,
                              normalizar_espacios=NORMALIZAR_ESPACIOS)

# Métricas: cuántas filas cambiaron
if COLS_A_LIMPIAR is None:
    before_txt = df.astype(str).agg(" ".join, axis=1)
    after_txt  = df_limpio.astype(str).agg(" ".join, axis=1)
else:
    before_txt = df[COLS_A_LIMPIAR].astype(str).agg(" ".join, axis=1)
    after_txt  = df_limpio[COLS_A_LIMPIAR].astype(str).agg(" ".join, axis=1)

cambiadas = (before_txt != after_txt)
print(f"Filas modificadas por la limpieza: {int(cambiadas.sum())} de {len(df)}")

# Guardar CSV limpio
df_limpio.to_csv(GUARDAR_LIMPIO_A, index=False)
print(f"Archivo limpio guardado en: {GUARDAR_LIMPIO_A}")


Filas modificadas por la limpieza: 19825 de 75112
Archivo limpio guardado en: datos_limpios.csv
